In [1]:
import tournament as tn
import numpy as np
import pandas as pd
import seaborn as sns
import glob


In [10]:
print(glob.glob("data/*.csv"))

['data/alta2016.csv', 'data/asu2016.csv', 'data/berk2016.csv', 'data/blake2016.csv', 'data/emery2016.csv', 'data/fullerton2016.csv', 'data/georgetown2016.csv', 'data/glenbrooks2016.csv', 'data/goldendesert2016.csv', 'data/gonzaga2016.csv', 'data/ndca2016.csv', 'data/toc2016.csv']


### Estimating real distributions of skill

We can find parameters that allow our distributions to fit the given data well. We do this via maximum likelihood estimation (MLE) and log-likelihood calculation, a procedure well-established in the literature.

Formally, we say:

$$LL_x = N_x \times \ln(P\hat (X=x))$$

and 

$$LL = \sum_{x} LL_x$

Here, $x$ represents the number of wins; i.e. between 0 and 6 in most specifications. Then, $N_x$ is the number of teams in the real tournament which have $x$ wins. 

We present a case using data from the 2016 [National Debate Coaches Association Championship](http://www.debatecoaches.org/). Here, 68 teams debated 6 rounds each. The distribution is shown below.

**CHART/HISTOGRAM GOES HERE**

Then, we choose to model team skill using a beta distribution. This distribution is well established in the literature for its flexibility and ease of computation. The beta distribution is defined on the interval [0, 1] and takes two shape parameters, which we will call $\alpha$ and $\beta$. Then, we perform optimization on $\alpha$ and $\beta$ to find the best combination of parameters which fits the observed data.

Although the hypergeometric distribution more closely resembles the 'story,' as a team has 6 chances to pick samples from 6 wins and 6 losses, the hypergeometric does not take into account the [...]



In [7]:
ndca_summ = pd.read_csv("data/ndca_summ.csv")
ndca_summ['count']

0     2
1     8
2    16
3    16
4    19
5     5
6     2
Name: count, dtype: int64

In [57]:
alpha_hat = 2.0
beta_hat = 5.0
ndca = tn.Simulation(68, 6, 100, **{'dist': 'beta', 'shape1': alpha_hat, 'shape2': beta_hat})
ndca.simulate()
y_hat = ndca.get_results().wins.value_counts(normalize=True, sort=True).sort_index()
logLikelihood = -sum(np.log(y_hat) * ndca_summ['count'])

In [13]:
def estimateLL(params):
    alpha_hat = params[0]
    beta_hat = params[1]
    t = tn.Simulation(68, 6, 20, **{'dist': 'beta', 'shape1': alpha_hat, 'shape2': beta_hat})
    t.simulate()
    y_hat = t.get_results().wins.value_counts(normalize=True, sort=True).sort_index()
    log_like = -sum(np.log(y_hat) * ndca_summ['count'])
    return log_like


In [11]:
from scipy.optimize import minimize

init_params = [2.0, 5.0]
# results = minimize(estimateLL, init_params, method='L-BFGS-B')


In [15]:
print(estimateLL([2.0,5.0]))
print(estimateLL([1.0,5.0]))
print(estimateLL([1.5,5.0]))
print(estimateLL([1.0,0.5]))
print(estimateLL([1.5,0.5]))
print(estimateLL([1.5,1.0]))
print(estimateLL([1.5,1.5]))

116.571463809
116.68325255
116.418556031
116.427861262
116.020138079
118.060026772
116.476572474


In [5]:
def estimateLLExp(params):
    alpha_hat = params[0]
    ndca = tn.Simulation(68, 6, 20, **{'dist': 'exp', 'sc': alpha_hat})
    ndca.simulate()
    y_hat = ndca.get_results().wins.value_counts(normalize=True, sort=True).sort_index()
    log_like = -sum(np.log(y_hat) * ndca_summ['count'])
    return log_like

init_params2 = [0.5]
# results2 = minimize(estimateLLExp, init_params2, method='L-BFGS-B')


In [10]:
print(estimateLLExp([0.1]))
print(estimateLLExp([0.3]))
print(estimateLLExp([0.5]))
print(estimateLLExp([0.7]))
print(estimateLLExp([1.0]))
print(estimateLLExp([1.5]))
print(estimateLLExp([2.0]))
print(estimateLLExp([2.5]))

120.37502362
121.058850217
119.623935197
117.987665343
120.897590488
120.909639898
119.90027864
118.657911614
